In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.layers import Dense, Input, concatenate, GRU, LSTM
from keras import backend as K
from keras.utils import to_categorical
from keras import Model
from nltk.translate.bleu_score import sentence_bleu
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [3]:
df = pd.DataFrame(X)

In [4]:
bin_labels = pd.read_csv('../../data/raw/rpart_bin_labels.csv', delimiter=",")

In [5]:
bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [6]:
bin_labels

,label,bins
0,A,0.80
1,B,1.35
2,C,1.55
3,D,1.65
4,E,1.75
5,F,1.85
6,G,2.45
7,H,2.55
8,I,2.65
9,J,2.85


In [7]:
# path_df = pd.read_csv('../../data/raw/rpart_paths.csv', delimiter=",")
# path_df = pd.read_csv('../../data/raw/test_paths.csv', delimiter=",")
path_df = pd.read_csv('../../data/raw/local_dt/local_paths.csv', delimiter=",")

In [8]:
path_df

,Unnamed: 0,matrix.ncol...1..nrow...nrow.iris..
0,1,1G0
1,2,3J0
2,3,1U0
3,4,1AC0
4,5,1N0
5,6,1AO0
6,7,2Y1
7,8,1G0
8,9,3AK0
9,10,1V0


In [9]:
path_df = path_df.drop(["Unnamed: 0"], axis=1)
path_df = path_df.rename(columns={list(path_df)[0]: "new_col"})

In [10]:
test_data = pd.concat([df, path_df], axis=1)
# test_data = df

In [11]:
test_data

,0,1,2,3,new_col
0,-0.900681,1.019004,-1.340227,-1.315444,1G0
1,-1.143017,-0.131979,-1.340227,-1.315444,3J0
2,-1.385353,0.328414,-1.397064,-1.315444,1U0
3,-1.506521,0.098217,-1.283389,-1.315444,1AC0
4,-1.021849,1.249201,-1.340227,-1.315444,1N0
5,-0.537178,1.939791,-1.169714,-1.052180,1AO0
6,-1.506521,0.788808,-1.340227,-1.183812,2Y1
7,-1.021849,0.788808,-1.283389,-1.315444,1G0
8,-1.748856,-0.362176,-1.340227,-1.315444,3AK0
9,-1.143017,0.098217,-1.283389,-1.447076,1V0


In [12]:
new_path = []
for i, val in test_data.iterrows():
    new_path.append(val['new_col'].split(sep=","))

In [13]:
test_data.head()

,0,1,2,3,new_col
0,-0.900681,1.019004,-1.340227,-1.315444,1G0
1,-1.143017,-0.131979,-1.340227,-1.315444,3J0
2,-1.385353,0.328414,-1.397064,-1.315444,1U0
3,-1.506521,0.098217,-1.283389,-1.315444,1AC0
4,-1.021849,1.249201,-1.340227,-1.315444,1N0


In [14]:
_ = [x.insert(0, 'S') for x in new_path]
_ = [x.append('E') for x in new_path]

In [15]:
new_path

[['S', '1G0', 'E'],
 ['S', '3J0', 'E'],
 ['S', '1U0', 'E'],
 ['S', '1AC0', 'E'],
 ['S', '1N0', 'E'],
 ['S', '1AO0', 'E'],
 ['S', '2Y1', 'E'],
 ['S', '1G0', 'E'],
 ['S', '3AK0', 'E'],
 ['S', '1V0', 'E'],
 ['S', '3R0', 'E'],
 ['S', '3J0', 'E'],
 ['S', '3R0', 'E'],
 ['S', '3BJ0', 'E'],
 ['S', '3Y0', 'E'],
 ['S', '3X0', 'E'],
 ['S', '1V0', 'E'],
 ['S', '3F0', '1BM0', 'E'],
 ['S', '1BU0', '3BR0', 'E'],
 ['S', '3F0', '1G0', 'E'],
 ['S', '1O0', 'E'],
 ['S', '3BJ0', 'E'],
 ['S', '1V0', '1AH0', 'E'],
 ['S', '1O0', 'E'],
 ['S', '3F0', '1AO0', 'E'],
 ['S', '3U0', '1AO0', 'E'],
 ['S', '3AK0', 'E'],
 ['S', '1AC0', 'E'],
 ['S', '3F0', '1G0', 'E'],
 ['S', '2E1', 'E'],
 ['S', '3BL0', '2AM1', 'E'],
 ['S', '3D0', 'E'],
 ['S', '3M0', '1G0', 'E'],
 ['S', '3F0', '2CN1', 'E'],
 ['S', '1M0', 'E'],
 ['S', '1BM0', 'E'],
 ['S', '3X0', 'E'],
 ['S', '3AS0', 'E'],
 ['S', '3AN0', '1BL0', 'E'],
 ['S', '3Y0', 'E'],
 ['S', '1CE0', 'E'],
 ['S', '1BF0', 'E'],
 ['S', '2E1', 'E'],
 ['S', '3X0', 'E'],
 ['S', '3BJ0', 'E'],


In [16]:
test_data['new_path'] = new_path

In [17]:
# 3G1', '4E0', '3R0', '4D0
# 3G1', '4E1', '3Q1
# a = [1,0,0,0]
# b = [1,1,1]
# c = [0]

In [18]:
test_data = test_data.drop(["new_col"], axis=1)

In [19]:
test_data.head()

,0,1,2,3,new_path
0,-0.900681,1.019004,-1.340227,-1.315444,"[S, 1G0, E]"
1,-1.143017,-0.131979,-1.340227,-1.315444,"[S, 3J0, E]"
2,-1.385353,0.328414,-1.397064,-1.315444,"[S, 1U0, E]"
3,-1.506521,0.098217,-1.283389,-1.315444,"[S, 1AC0, E]"
4,-1.021849,1.249201,-1.340227,-1.315444,"[S, 1N0, E]"


In [20]:
paths_lengths = np.array([len(xi)
                          for xi in test_data.iloc[:,-1]])

In [21]:
paths_lengths
np.max(paths_lengths)

5

In [22]:
label_char = []
for _, i in enumerate(np.unique(test_data['new_path'])):
    for _, j in enumerate(i):
        if j not in label_char:
            label_char.append(j)

In [23]:
label_indices = { j : i for i, j in enumerate(label_char) }

In [24]:
label_indices

{'S': 0,
 '1AC0': 1,
 'E': 2,
 '1AC1': 3,
 '1AZ0': 4,
 '1BN1': 5,
 '3AY1': 6,
 '3BT0': 7,
 '1AD0': 8,
 '3R0': 9,
 '1AO0': 10,
 '1AO1': 11,
 '1BW0': 12,
 '1DG0': 13,
 '2J0': 14,
 '1P1': 15,
 '3CX1': 16,
 '3U1': 17,
 '1AP0': 18,
 '1CH1': 19,
 '1AZ1': 20,
 '1AV1': 21,
 '1BF0': 22,
 '1BM0': 23,
 '1BN0': 24,
 '1BX1': 25,
 '1BU0': 26,
 '3BR0': 27,
 '1BU1': 28,
 '1CE0': 29,
 '1CE1': 30,
 '1CL0': 31,
 '3AH1': 32,
 '3CX0': 33,
 '1CX1': 34,
 '1DI1': 35,
 '1G0': 36,
 '1G1': 37,
 '2AM0': 38,
 '3BL0': 39,
 '1H1': 40,
 '1M0': 41,
 '1N0': 42,
 '1N1': 43,
 '1AV0': 44,
 '1O0': 45,
 '1O1': 46,
 '1DI0': 47,
 '2AM1': 48,
 '3AH0': 49,
 '3CF0': 50,
 '3CH0': 51,
 '1U0': 52,
 '1V0': 53,
 '1AH0': 54,
 '1V1': 55,
 '3CI0': 56,
 '3F1': 57,
 '2AG0': 58,
 '3T1': 59,
 '1DS1': 60,
 '2AT0': 61,
 '1CW0': 62,
 '3T0': 63,
 '4AQ1': 64,
 '2BD0': 65,
 '1CY0': 66,
 '1DF0': 67,
 '2E0': 68,
 '2E1': 69,
 '2L0': 70,
 '1DM1': 71,
 '2X0': 72,
 '2Y1': 73,
 '1BF1': 74,
 '3AB0': 75,
 '3AK0': 76,
 '3AK1': 77,
 '1DM0': 78,
 '3AL1': 79,

In [25]:
len(label_indices)

117

In [26]:
indices_label = { i : j for i, j in enumerate(label_char) }
indices_label

{0: 'S',
 1: '1AC0',
 2: 'E',
 3: '1AC1',
 4: '1AZ0',
 5: '1BN1',
 6: '3AY1',
 7: '3BT0',
 8: '1AD0',
 9: '3R0',
 10: '1AO0',
 11: '1AO1',
 12: '1BW0',
 13: '1DG0',
 14: '2J0',
 15: '1P1',
 16: '3CX1',
 17: '3U1',
 18: '1AP0',
 19: '1CH1',
 20: '1AZ1',
 21: '1AV1',
 22: '1BF0',
 23: '1BM0',
 24: '1BN0',
 25: '1BX1',
 26: '1BU0',
 27: '3BR0',
 28: '1BU1',
 29: '1CE0',
 30: '1CE1',
 31: '1CL0',
 32: '3AH1',
 33: '3CX0',
 34: '1CX1',
 35: '1DI1',
 36: '1G0',
 37: '1G1',
 38: '2AM0',
 39: '3BL0',
 40: '1H1',
 41: '1M0',
 42: '1N0',
 43: '1N1',
 44: '1AV0',
 45: '1O0',
 46: '1O1',
 47: '1DI0',
 48: '2AM1',
 49: '3AH0',
 50: '3CF0',
 51: '3CH0',
 52: '1U0',
 53: '1V0',
 54: '1AH0',
 55: '1V1',
 56: '3CI0',
 57: '3F1',
 58: '2AG0',
 59: '3T1',
 60: '1DS1',
 61: '2AT0',
 62: '1CW0',
 63: '3T0',
 64: '4AQ1',
 65: '2BD0',
 66: '1CY0',
 67: '1DF0',
 68: '2E0',
 69: '2E1',
 70: '2L0',
 71: '1DM1',
 72: '2X0',
 73: '2Y1',
 74: '1BF1',
 75: '3AB0',
 76: '3AK0',
 77: '3AK1',
 78: '1DM0',
 79: '3AL1',

In [27]:
bin_labels

,label,bins
0,A,0.80
1,B,1.35
2,C,1.55
3,D,1.65
4,E,1.75
5,F,1.85
6,G,2.45
7,H,2.55
8,I,2.65
9,J,2.85


In [28]:
np.unique(path_df, return_counts=True)

(array(['1AC0', '1AC1,1AZ0', '1AC1,1BN1', '1AC1,3AY1', '1AC1,3BT0',
        '1AD0,3R0', '1AO0', '1AO1', '1AO1,1BW0', '1AO1,1DG0,2J0',
        '1AO1,1P1', '1AO1,3CX1', '1AO1,3U1', '1AP0,1CH1', '1AZ1,1AV1',
        '1AZ1,3U1', '1BF0', '1BM0', '1BN0,1BX1', '1BU0,3BR0', '1BU1',
        '1BX1', '1CE0', '1CE1,1CL0', '1CE1,3AH1', '1CE1,3CX0', '1CE1,3CX1',
        '1CX1,1AZ0', '1DI1', '1G0', '1G1,2AM0', '1G1,3BL0', '1H1',
        '1H1,3BT0', '1M0', '1N0', '1N1,1AV0', '1N1,1AV1', '1O0',
        '1O1,1DI0', '1O1,2AM1', '1O1,3AH0', '1O1,3CF0', '1O1,3CH0', '1P1',
        '1U0', '1V0', '1V0,1AH0', '1V1,3CI0', '1V1,3CX0', '1V1,3F1',
        '2AG0,3T1', '2AM0', '2AM0,1DS1', '2AT0,1CW0', '2AT0,3T0',
        '2AT0,4AQ1,1AD0', '2BD0,1CY0', '2BD0,1DF0', '2E0', '2E0,3U1',
        '2E1', '2L0,1DM1', '2X0', '2Y1', '2Y1,1BF1', '3AB0,1AC1', '3AK0',
        '3AK1,1DM0', '3AK1,3CX0', '3AL1,1AD0', '3AL1,3CH1', '3AN0,1AB1',
        '3AN0,1BL0', '3AN0,1CF1', '3AS0', '3AY0', '3AY1', '3BJ0',
        '3BJ1,1DJ0', '3B

In [29]:
path_df

,new_col
0,1G0
1,3J0
2,1U0
3,1AC0
4,1N0
5,1AO0
6,2Y1
7,1G0
8,3AK0
9,1V0


In [30]:
input_path_sequence = []
next_chars = []
features = []
paths_maxlen = np.max(paths_lengths)
# path_vocab_size = len(bin_labels) # How is this working? Validate!
path_vocab_size = len(indices_label) # Temporary test for local trees
feature_size = 4
for i in range(0, len(test_data)):
    # get the feature
    curr_feat = np.array([test_data.iloc[i, 0:4]])
    curr_path = test_data.iloc[i, -1]
    curr_path_len = len(curr_path)
    # curr_label = y[i]
    # curr_dec_feat = df.iloc[i, 6]
    for j in range(1, curr_path_len):
        features.append(curr_feat)
        input_path_sequence.append(curr_path[0:j])
        next_chars.append(curr_path[j])

x_path = np.zeros(
    (len(input_path_sequence), paths_maxlen, path_vocab_size), dtype=np.bool)

path_latent_input = np.zeros(
    (len(input_path_sequence), feature_size), dtype=np.float)

y_path = np.zeros(
    (len(input_path_sequence), path_vocab_size), dtype=np.bool)

# print(input_path_sequence)
# print(len(input_path_sequence))
for i, sentence in enumerate(input_path_sequence):
    for t, char in enumerate(sentence):
        # x_path[i, t, self.char_indices[char]] = 1
        # print(bin_labels.index[bin_labels['label'] == char[1]])
        # index = bin_labels.index[bin_labels['label'] == char[1]].tolist()[0]
        x_path[i, t, label_indices[char]] = 1
    # y_path[i, char_indices[next_chars[i]]] = 1
    # index = bin_labels.index[bin_labels['label'] == next_chars[i][1]].tolist()[0]
    # y_path[i, index] = 1
    y_path[i, label_indices[next_chars[i]]] = 1
    path_latent_input[i, :] = features[i]
    
## Trouble with "S" and "E" index values.

In [31]:
len(input_path_sequence)

381

In [32]:
len(next_chars)

381

In [33]:
y_path.shape

(381, 117)

In [34]:
from keras.layers import Dropout

In [35]:
def _create_label_model(latent_dim=5):
    input_layer = Input(shape=(feature_size,), name='ip_x')
    hidden_layer_x1 = Dense(20, activation='tanh',
                            name='hidden_x1')(input_layer)
    hidden_layer_x2 = Dense(20, activation='tanh',
                            name='hidden_x2')(hidden_layer_x1)
    hidden_layer_x3 = Dense(latent_dim, activation='tanh',
                            name='hidden_x3')(hidden_layer_x2)
    output_layer = Dense(len(np.unique(y)), activation='softmax',
                         name='op_x')(hidden_layer_x3)
    model = Model(input_layer, output_layer)
    return model

def _create_combined_model(initialize=True, rnn_cell='gru', latent_dim=5):

    label_model_latent = Input(shape=(latent_dim,), name='label_ip')
    path_input = Input(shape=(
        paths_maxlen, path_vocab_size), name='dec_feat_ip')
    if rnn_cell == 'gru':
        RNN = GRU
    else:
        RNN = LSTM

    decoder = RNN(latent_dim, return_state=False,
                  return_sequences=False, name='gru_seq')
                 # dropout=0.5, recurrent_dropout=0.5)
    if initialize:
        decoder_outputs = decoder(
            path_input, initial_state=label_model_latent)
    else:
        decoder_outputs = decoder(path_input)
    merge_layer = concatenate([label_model_latent, decoder_outputs], name='cat')
#     merge_layer = concatenate([label_model_latent, dropout_layer], name='cat')
    dropout_layer = Dropout(0.2)(merge_layer)
    output_chars = Dense(path_vocab_size,
                         activation='softmax', name='op_sent')(dropout_layer)
    model = Model(
        [label_model_latent, path_input], output_chars)
    return model

In [36]:
combined_model = _create_combined_model()
label_model = _create_label_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [37]:
def get_hidden_x(x, model, layer_num=3):
    def get_hidden_x_inner(model, layer_num=layer_num):
        return K.function([model.layers[0].input], [model.layers[layer_num].output])
    return get_hidden_x_inner(model, layer_num=layer_num)([x])[0]

In [38]:
def fit_model():

    y_cat = to_categorical(y)

    label_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    label_model.fit(
        X, y_cat, batch_size=30, epochs=150, verbose=0, shuffle=True, validation_split=0.2)

    x_latent = get_hidden_x(path_latent_input, model=label_model)

    combined_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    combined_model.fit([x_latent, x_path], y_path,
                           batch_size=30, epochs=5000, verbose=1, shuffle=True)

In [39]:
fit_model()

Instructions for updating:
Use tf.cast instead.
Epoch 1/5000
381/381 [==============================] - 0s 1ms/step - loss: 4.7720 - acc: 0.0000e+00
Epoch 2/5000
381/381 [==============================] - 0s 131us/step - loss: 4.7468 - acc: 0.0079
Epoch 3/5000
381/381 [==============================] - 0s 126us/step - loss: 4.7175 - acc: 0.0131
Epoch 4/5000
381/381 [==============================] - 0s 103us/step - loss: 4.6788 - acc: 0.0105
Epoch 5/5000
381/381 [==============================] - 0s 119us/step - loss: 4.6326 - acc: 0.0289
Epoch 6/5000
381/381 [==============================] - 0s 114us/step - loss: 4.6030 - acc: 0.0499
Epoch 7/5000
381/381 [==============================] - 0s 118us/step - loss: 4.5729 - acc: 0.0787
Epoch 8/5000
381/381 [==============================] - 0s 125us/step - loss: 4.5327 - acc: 0.1391
Epoch 9/5000
381/381 [==============================] - 0s 112us/step - loss: 4.4774 - acc: 0.1654
Epoch 10/5000
381/381 [==============================] - 0s

381/381 [==============================] - 0s 130us/step - loss: 3.0704 - acc: 0.3963
Epoch 81/5000
381/381 [==============================] - 0s 106us/step - loss: 3.0259 - acc: 0.3937
Epoch 82/5000
381/381 [==============================] - 0s 122us/step - loss: 3.0550 - acc: 0.3990
Epoch 83/5000
381/381 [==============================] - 0s 132us/step - loss: 3.0349 - acc: 0.3937
Epoch 84/5000
381/381 [==============================] - 0s 124us/step - loss: 3.0583 - acc: 0.3911
Epoch 85/5000
381/381 [==============================] - 0s 138us/step - loss: 3.0689 - acc: 0.4016
Epoch 86/5000
381/381 [==============================] - 0s 124us/step - loss: 3.0844 - acc: 0.3937
Epoch 87/5000
381/381 [==============================] - 0s 124us/step - loss: 3.0278 - acc: 0.3911
Epoch 88/5000
381/381 [==============================] - 0s 147us/step - loss: 3.0043 - acc: 0.3937
Epoch 89/5000
381/381 [==============================] - 0s 130us/step - loss: 3.0253 - acc: 0.3937
Epoch 90/5000


381/381 [==============================] - 0s 153us/step - loss: 2.6879 - acc: 0.4042
Epoch 162/5000
381/381 [==============================] - 0s 131us/step - loss: 2.6850 - acc: 0.4094
Epoch 163/5000
381/381 [==============================] - 0s 122us/step - loss: 2.6838 - acc: 0.4121
Epoch 164/5000
381/381 [==============================] - 0s 109us/step - loss: 2.7113 - acc: 0.4094
Epoch 165/5000
381/381 [==============================] - 0s 106us/step - loss: 2.6899 - acc: 0.4147
Epoch 166/5000
381/381 [==============================] - 0s 110us/step - loss: 2.6691 - acc: 0.4173
Epoch 167/5000
381/381 [==============================] - 0s 160us/step - loss: 2.6616 - acc: 0.4147
Epoch 168/5000
381/381 [==============================] - 0s 114us/step - loss: 2.6405 - acc: 0.4199
Epoch 169/5000
381/381 [==============================] - 0s 111us/step - loss: 2.6088 - acc: 0.4121
Epoch 170/5000
381/381 [==============================] - 0s 127us/step - loss: 2.6229 - acc: 0.4278
Epoch

381/381 [==============================] - 0s 124us/step - loss: 2.4363 - acc: 0.4252
Epoch 243/5000
381/381 [==============================] - 0s 109us/step - loss: 2.4812 - acc: 0.4278
Epoch 244/5000
381/381 [==============================] - 0s 111us/step - loss: 2.4595 - acc: 0.4409
Epoch 245/5000
381/381 [==============================] - 0s 125us/step - loss: 2.4545 - acc: 0.4357
Epoch 246/5000
381/381 [==============================] - 0s 99us/step - loss: 2.4421 - acc: 0.4199
Epoch 247/5000
381/381 [==============================] - 0s 124us/step - loss: 2.4566 - acc: 0.4304
Epoch 248/5000
381/381 [==============================] - 0s 116us/step - loss: 2.4439 - acc: 0.4409
Epoch 249/5000
381/381 [==============================] - 0s 101us/step - loss: 2.4459 - acc: 0.4357
Epoch 250/5000
381/381 [==============================] - 0s 116us/step - loss: 2.4542 - acc: 0.4278
Epoch 251/5000
381/381 [==============================] - 0s 126us/step - loss: 2.4569 - acc: 0.4304
Epoch 

381/381 [==============================] - 0s 127us/step - loss: 2.3868 - acc: 0.4383
Epoch 324/5000
381/381 [==============================] - 0s 124us/step - loss: 2.3711 - acc: 0.4383
Epoch 325/5000
381/381 [==============================] - 0s 125us/step - loss: 2.3750 - acc: 0.4357
Epoch 326/5000
381/381 [==============================] - 0s 137us/step - loss: 2.3619 - acc: 0.4436
Epoch 327/5000
381/381 [==============================] - 0s 135us/step - loss: 2.3597 - acc: 0.4278
Epoch 328/5000
381/381 [==============================] - 0s 126us/step - loss: 2.3660 - acc: 0.4409
Epoch 329/5000
381/381 [==============================] - 0s 129us/step - loss: 2.3678 - acc: 0.4304
Epoch 330/5000
381/381 [==============================] - 0s 123us/step - loss: 2.3620 - acc: 0.4252
Epoch 331/5000
381/381 [==============================] - 0s 133us/step - loss: 2.3541 - acc: 0.4462
Epoch 332/5000
381/381 [==============================] - 0s 124us/step - loss: 2.3663 - acc: 0.4357
Epoch

...

381/381 [==============================] - 0s 114us/step - loss: 1.7104 - acc: 0.5302
Epoch 4652/5000
381/381 [==============================] - 0s 107us/step - loss: 1.7365 - acc: 0.5066
Epoch 4653/5000
381/381 [==============================] - 0s 99us/step - loss: 1.7706 - acc: 0.5223
Epoch 4654/5000
381/381 [==============================] - 0s 114us/step - loss: 1.7581 - acc: 0.4987
Epoch 4655/5000
381/381 [==============================] - 0s 103us/step - loss: 1.7093 - acc: 0.5197
Epoch 4656/5000
381/381 [==============================] - 0s 106us/step - loss: 1.7337 - acc: 0.5118
Epoch 4657/5000
381/381 [==============================] - 0s 114us/step - loss: 1.7412 - acc: 0.5223
Epoch 4658/5000
381/381 [==============================] - 0s 104us/step - loss: 1.6857 - acc: 0.5328
Epoch 4659/5000
381/381 [==============================] - 0s 103us/step - loss: 1.7397 - acc: 0.5328
Epoch 4660/5000
381/381 [==============================] - 0s 124us/step - loss: 1.7309 - acc: 0.51

381/381 [==============================] - 0s 106us/step - loss: 1.7550 - acc: 0.5039
Epoch 4732/5000
381/381 [==============================] - 0s 96us/step - loss: 1.7557 - acc: 0.5144
Epoch 4733/5000
381/381 [==============================] - 0s 103us/step - loss: 1.6800 - acc: 0.5433
Epoch 4734/5000
381/381 [==============================] - 0s 108us/step - loss: 1.7302 - acc: 0.5223
Epoch 4735/5000
381/381 [==============================] - 0s 98us/step - loss: 1.7234 - acc: 0.5328
Epoch 4736/5000
381/381 [==============================] - 0s 110us/step - loss: 1.7114 - acc: 0.5223
Epoch 4737/5000
381/381 [==============================] - 0s 105us/step - loss: 1.7332 - acc: 0.5144
Epoch 4738/5000
381/381 [==============================] - 0s 99us/step - loss: 1.7215 - acc: 0.5118
Epoch 4739/5000
381/381 [==============================] - 0s 121us/step - loss: 1.6885 - acc: 0.5302
Epoch 4740/5000
381/381 [==============================] - 0s 112us/step - loss: 1.7233 - acc: 0.5328

381/381 [==============================] - 0s 111us/step - loss: 1.7602 - acc: 0.5144
Epoch 4812/5000
381/381 [==============================] - 0s 114us/step - loss: 1.6953 - acc: 0.5249
Epoch 4813/5000
381/381 [==============================] - 0s 103us/step - loss: 1.7255 - acc: 0.5354
Epoch 4814/5000
381/381 [==============================] - 0s 106us/step - loss: 1.7738 - acc: 0.5013
Epoch 4815/5000
381/381 [==============================] - 0s 103us/step - loss: 1.7212 - acc: 0.5013
Epoch 4816/5000
381/381 [==============================] - 0s 99us/step - loss: 1.7386 - acc: 0.5249
Epoch 4817/5000
381/381 [==============================] - 0s 104us/step - loss: 1.7221 - acc: 0.5249
Epoch 4818/5000
381/381 [==============================] - 0s 98us/step - loss: 1.7695 - acc: 0.5302
Epoch 4819/5000
381/381 [==============================] - 0s 121us/step - loss: 1.7074 - acc: 0.5092
Epoch 4820/5000
381/381 [==============================] - 0s 121us/step - loss: 1.6828 - acc: 0.524

381/381 [==============================] - 0s 103us/step - loss: 1.7750 - acc: 0.5144
Epoch 4892/5000
381/381 [==============================] - 0s 110us/step - loss: 1.6983 - acc: 0.5171
Epoch 4893/5000
381/381 [==============================] - 0s 97us/step - loss: 1.7325 - acc: 0.5223
Epoch 4894/5000
381/381 [==============================] - 0s 106us/step - loss: 1.7479 - acc: 0.5092
Epoch 4895/5000
381/381 [==============================] - 0s 104us/step - loss: 1.7146 - acc: 0.5302
Epoch 4896/5000
381/381 [==============================] - 0s 109us/step - loss: 1.7204 - acc: 0.5013
Epoch 4897/5000
381/381 [==============================] - 0s 112us/step - loss: 1.7542 - acc: 0.5092
Epoch 4898/5000
381/381 [==============================] - 0s 94us/step - loss: 1.6997 - acc: 0.5092
Epoch 4899/5000
381/381 [==============================] - 0s 106us/step - loss: 1.7197 - acc: 0.5302
Epoch 4900/5000
381/381 [==============================] - 0s 102us/step - loss: 1.6987 - acc: 0.517

381/381 [==============================] - 0s 113us/step - loss: 1.7450 - acc: 0.5171
Epoch 4972/5000
381/381 [==============================] - 0s 100us/step - loss: 1.7058 - acc: 0.5039
Epoch 4973/5000
381/381 [==============================] - 0s 98us/step - loss: 1.7871 - acc: 0.5144
Epoch 4974/5000
381/381 [==============================] - 0s 100us/step - loss: 1.6906 - acc: 0.5381
Epoch 4975/5000
381/381 [==============================] - 0s 104us/step - loss: 1.7278 - acc: 0.4961
Epoch 4976/5000
381/381 [==============================] - 0s 101us/step - loss: 1.7350 - acc: 0.5197
Epoch 4977/5000
381/381 [==============================] - 0s 99us/step - loss: 1.7092 - acc: 0.5249
Epoch 4978/5000
381/381 [==============================] - 0s 101us/step - loss: 1.7266 - acc: 0.5066
Epoch 4979/5000
381/381 [==============================] - 0s 101us/step - loss: 1.7002 - acc: 0.5171
Epoch 4980/5000
381/381 [==============================] - 0s 101us/step - loss: 1.7055 - acc: 0.530

In [54]:
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
label_ip (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
dec_feat_ip (InputLayer)        (None, 5, 117)       0                                            
__________________________________________________________________________________________________
gru_seq (GRU)                   (None, 5)            1845        dec_feat_ip[0][0]                
                                                                 label_ip[0][0]                   
__________________________________________________________________________________________________
cat (Concatenate)               (None, 10)           0           label_ip[0][0]                   
          

In [43]:
def predict(x):
    latent_dim = 5
    x_f = x.reshape(1, feature_size)
    token = 'S'
    cont = True
    path = [token]
    x_path = np.zeros((1, paths_maxlen, path_vocab_size), dtype=np.bool)

    x_latent = get_hidden_x(x_f, model=label_model)
    x_latent = x_latent.reshape(1, latent_dim)
    x_path[0, 0, label_indices[token]] = 1
    pred = label_model.predict(x_f)
    label = [np.argmax(pred[0])]
    index = 1
    while cont & (index < paths_maxlen):
        pred = combined_model.predict([x_latent, x_path])
        char_index = np.argmax(pred[0])
        x_path[0, index, char_index] = 1
        next_char = indices_label[char_index]
        path.append(next_char)
        index += 1
        if next_char == 'E':
            cont = False
        # elif index == self.paths_maxlen - 1:
        #     path.append('E')

    if path[-1] != 'E':
        path.append('E')

    return [path, label]

In [42]:
def check_path(path): # Returns -1 if path traversed is wrong/non-existant
    # path = ''.join(path)
    path = path[1:-1]
    pred_features = []
    path_as_strings = []
    for i in range(len(path)):
        pred_features.append(int(path[i][:-1]))
        path_as_strings.append(path[i][-1])
        # if i%2 == 0:
        #     print('i -- ', i)
        #     print('path -- ', path)
        #     print('path[i] -- ', path[i])
        #     pred_features.append(int(path[i]))
        # else:
        #     path_as_strings.append(path[i])

    n_nodes = self.clf.tree_.node_count
    children_left = self.clf.tree_.children_left
    children_right = self.clf.tree_.children_right
    feature = self.clf.tree_.feature

    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        # node_depth[node_id] = parent_depth + 1

        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True


    node = 0
    pred_target = -1
    subset_path = False
    for i in range(len(path_as_strings)):
        if path_as_strings[i] == 'L':
            if feature[node]+1 == pred_features[i]:
                node = children_left[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        elif path_as_strings[i] == 'R':
            if feature[node]+1 == pred_features[i]:
                node = children_right[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        if is_leaves[node]:
            for i, x in enumerate(self.clf.tree_.value[node][0]):
                if x > 0:
                    pred_target = i
            if i < len(path_as_strings):
                subset_path = True

    return pred_target, subset_path

In [44]:
def score():
    count = []
    bleu_score = []
    j_coeff = []
    l_dist = []
    path_mismatch_count = []
    traverse_check_count = []
    order_mismatch_count = []
    subset_path_count = []
    for i in range(test_data.shape[0]):
        curr_feat = np.array([test_data.iloc[i, 0:X.shape[1]]])
        path, label = predict(curr_feat)
        actual_path = test_data.iloc[i, -1]

        actual_path_tok = [label_indices[char] for char in actual_path]
        pred_path_tok = [label_indices[char] for char in path]

        # j_coeff.append(super().get_j_coeff(actual_path_tok, pred_path_tok))

        print('actual vs predicted: ', test_data.iloc[i, -1], ' vs ', ' '.join(
            path), 'labels: ', y[i], label[0])
        count.append(y[i] == label[0])
        # print('Actual path -- ', actual_path)
        # print('Pred path -- ', path)
#         if actual_path != path:
#             print(' -- Path mismatch -- ')
#             if sorted(actual_path) == sorted(path):
#                 print(' -- Order mismatch -- ')
#                 order_mismatch_count.append(1)
#             else:
#                 path_mismatch_count.append(1)
#                 pred_target, subset_path = self.check_path(path)
#                 subset_path_count.append(subset_path)
#                 if pred_target != -1 and pred_target == self.df.iloc[i, self.X.shape[1]+1]:
#                     traverse_check_count.append(1)


        path = list(''.join(path))
        actual_path = list(''.join(test_data.iloc[i, -1]))
        bleu_score.append(sentence_bleu([actual_path], path))

#         lev_path = []
#         for i in range(len(path)):
#             if i in ['S','L','R','E']:
#                 lev_path.append(i)
#         l_dist.append(distance.levenshtein(
#             self.df.iloc[i, self.X.shape[1]].replace(' ', ''), ''.join(lev_path)))


    print('\nLabel accuracy - ', np.mean(count))
#     print('Path metric (Jaccard) - ', np.mean(j_coeff))
#     print('Path metric (Levenshtein) - ', np.mean(l_dist))
#     print('Path mismatch count - ', np.sum(path_mismatch_count))
#     print('Right traverse count - ', np.sum(traverse_check_count))
#     print('Order mismatch count - ', np.sum(order_mismatch_count))
#     print('Subset path count - ', np.sum(subset_path_count))
    print('Bleu score of paths - ', np.mean(bleu_score))

In [45]:
score()

actual vs predicted:  ['S', '1G0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3J0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1U0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1AC0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1N0', 'E']  vs  S E labels:  0 0


/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

actual vs predicted:  ['S', '1AO0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '2Y1', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1G0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3AK0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1V0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3R0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3J0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3R0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3BJ0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3Y0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3X0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1V0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3F0', '1BM0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1BU0', '3BR0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '3F0', '1G0', 'E']  vs  S E labels:  0 0
actual vs predicted:  ['S', '1O0', 'E']  vs

actual vs predicted:  ['S', '1DI1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3BR1', '1AV1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3T1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1CE1', '3AH1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '4BG1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1N1', '1AV1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '2AG0', '3T1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3S1', '3T1', 'E']  vs  S E labels:  2 1
actual vs predicted:  ['S', '1P1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '4AQ1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '4AQ1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1CE1', '3CX1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1AO1', '3U1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '1AZ1', '3U1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '4BZ1', 'E']  vs  S E labels:  2 2

### Path invariance trials -- generic

In [46]:
## Import nnum, vnum, nodes, csplit, split_df,
## frame

splits = pd.read_csv('../../data/raw/splits.csv', delimiter=",", index_col=0)
csplit = pd.read_csv('../../data/raw/csplit.csv', delimiter=",")
frame = pd.read_csv('../../data/raw/frame.csv', delimiter=",",index_col=0)

# frame = frame.drop(["Unnamed: 0"], axis=1)
frame = frame.rename(columns={"var": "variable"})
# bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [47]:
frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob
1,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000
2,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333
3,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667
6,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000
12,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000
24,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333
25,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667
13,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000
26,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000
52,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333


In [59]:
## Generate nnum, vnum, nodes(split_df and csplit - 2L if necessary)

temp_frame = frame

nc = temp_frame[["ncompete", "nsurrogate"]]

index = np.cumsum((frame[["variable"]]!="<leaf>").values + nc[["ncompete"]].values + nc[["nsurrogate"]].values)

index_df = pd.DataFrame((np.insert(index,0,0)+1)[:-1], columns=["i"], index=frame.index)

temp_frame = pd.concat([temp_frame, index_df], axis=1)

# temp_frame[temp_frame[["var"]]=="<leaf>"]
# temp_frame.where(temp_frame[["var"]]=="<leaf>")
# temp_frame.loc[temp_frame[["variable"]]=="<leaf>", "index"] = 0
temp_frame.i[temp_frame.variable == "<leaf>"] = 0

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [60]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
1,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
2,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
3,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
6,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
12,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
24,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
25,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
13,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
26,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
52,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [61]:
nodes = temp_frame[["n", "ncompete", "nsurrogate", "i"]]

In [62]:
nnum = list(temp_frame.index) # row names of temp_frame

In [63]:
feature_names = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]

vnum = list(map(feature_names.index, splits.index))

In [64]:
nodes

,n,ncompete,nsurrogate,i
1,150,3,3,1
2,50,0,0,0
3,100,3,3,8
6,54,3,0,15
12,48,3,0,19
24,47,0,0,0
25,1,0,0,0
13,6,3,2,23
26,3,3,0,29
52,2,0,0,0


In [80]:
nnum

[1, 2, 3, 6, 12, 24, 25, 13, 26, 52, 53, 27, 7, 14, 28, 29, 15]

In [81]:
def return_yval(path): # [1,0,0,0]
    node = 0
    nspl = 1
    i = 0
    while nspl != 0:
        npos = nnum[node] # i)0, 
        nspl = nodes.iloc[npos-1][3] # i)1
        var = vnum[nspl]
        # ncat
        temp = splits.iloc[nspl][3]
        if nspl > 0:
            print("nspl succeeded")
            if path[i] == 0: # i)1
                direction = -1
                i+=1
            else:
                direction = 1
                i+=1

            if direction == -1:
                print(node)
                if node == 0:
                    node = 1
                node = 2 * node
            else:
                print(node)
                if node == 0:
                    node = 1
                node = 2 * node + 1
            if len(path) == 1:
                nspl = 0
        else:
            print('nspl failed')
            print("leaf node -- ", node)
            yval = temp_frame.iloc[node][4]
    return temp_frame.iloc[node][4]

In [77]:
nodes.iloc[0]

n             150
ncompete        3
nsurrogate      3
i               1
Name: 1, dtype: int64

In [86]:

path = [1,1,0,1]
a = [1,0,0,0]
b = [1,1,1]
c = [0]
d = [1,1,1,1,1,1,0]
e = [1,0,1,0,0]
return_yval(e)

nspl succeeded
0
nspl failed
leaf node --  3


2

In [30]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
0,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
1,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
2,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
3,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
4,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
5,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
6,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
7,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
8,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
9,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [33]:
splits

,count,ncat,improve,index,adj
Petal.Length,150,-1,50.000000,2.45,0.000000
Petal.Width,150,-1,50.000000,0.80,0.000000
Sepal.Length,150,-1,34.164050,5.45,0.000000
Sepal.Width,150,1,19.038508,3.35,0.000000
Petal.Width,0,-1,1.000000,0.80,1.000000
Sepal.Length,0,-1,0.920000,5.45,0.760000
Sepal.Width,0,1,0.833333,3.35,0.500000
Petal.Width,100,-1,38.969404,1.75,0.000000
Petal.Length,100,-1,37.353535,4.75,0.000000
Sepal.Length,100,-1,10.686869,6.15,0.000000


In [96]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')